In [7]:
# =========================================================
# Imports
# =========================================================
import re
from IPython.display import display, Markdown
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM



### Splitting Documents into Chunks

1. When we read an entire PDF, we often get very large blocks of text — sometimes thousands of words per page. But LLMs (and embedding models) have context limits — they can’t process all that at once.

2. That’s why we split long documents into smaller, overlapping pieces, called chunks.

3. Each chunk will later become:
   - an embedding (vector representation)
   - a retrieval unit (the part that gets fetched when the user asks a question)

In [8]:

# Load PDF
loader = PyPDFLoader("/Users/swatichandna/SynologyDrive/GitHub/NLP/Module 10/ToyData/test.pdf")
raw_docs = loader.load()
print(f"Loaded {len(raw_docs)} pages")



# Chunk documents
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=120,
    separators=["\n\n", "\n", ".", " "]
)
chunks = splitter.split_documents(raw_docs)
print(f"Created {len(chunks)} chunks")


Loaded 598 pages
Created 1134 chunks


### Create Embeddings for Each Chunk

Each chunk of text  previous step must be converted into a vector — a list of numbers that captures its meaning.
This process is called embedding.

Why?
1. Machines can’t understand text, but they can compare numbers.
2. Each vector encodes the semantic meaning of a chunk.
3. Similar meanings → vectors are close together in multi-dimensional space.
4. So when a user asks a question, the query is also embedded, and we find the chunks whose vectors are most similar to it (via cosine similarity or L2 distance).

In [9]:

# Create embeddings + FAISS retriever
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
print("FAISS retriever ready")



# Helper to extract clean text
def get_context_text(query):
    results = retriever.invoke(query)
    if not results:
        return "No relevant context found."
    if hasattr(results[0], "page_content"):
        texts = [doc.page_content for doc in results]
    else:
        texts = [str(doc) for doc in results]
    return "\n\n".join(texts)


FAISS retriever ready


In [10]:

# Hugging Face LLM setup
model_id = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto")

hf_gen = pipeline(
    task="text-generation", #goal is to predict or generate text continuations
    model=model,  #Passes in your already loaded language model
    tokenizer=tokenizer,  #The tokenizer converts between text and token IDs
    max_new_tokens=200, #maximum number of tokens the model is allowed to generate
    temperature=0.3, #Controls the randomness of predictions,Lower = more deterministic, Higher = more creative/random.
    do_sample=False, #sample probabilistically or pick the top-scoring tokens deterministically.False means greedy decoding — always choose the most likely next token (no randomness).  
    top_p=0.9,  #nucleus sampling (a method to pick from only the most probable subset of tokens that together make up 90% of the probability mass).
    top_k=50, #Used for top-k sampling, which restricts the model to choose from the top 50 most likely next tokens.
    repetition_penalty=1.1, #Penalizes repeating tokens by slightly reducing their probability each time they appear.
    truncation=True, #input text is truncated if it exceeds the model’s maximum context length
    pad_token_id=tokenizer.eos_token_id, #padding token ID to use for sequences shorter than the model’s context length.
)



Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 20.80it/s]
Device set to use mps:0


In [11]:
#Build prompt and RAG chain

llm = HuggingFacePipeline(pipeline=hf_gen)
print("Hugging Face model ready")

# 
# templated prompt with two required variables: {context} and {question}
# At runtime, LangChain will format this template by substituting the actual context text and the user’s question.
prompt = ChatPromptTemplate.from_template("""
Use the following context to answer the user's question.

Context:
{context}

Question:
{question}

Answer clearly and concisely.
""")

# Build RAG chain
rag_chain = (
    {
        "context": lambda q: get_context_text(q), #Takes the incoming input q and calls your function get_context_text(q) to retrieve retrieved documents / context text
        "question": RunnablePassthrough(), #original input unchanged and stores it under the
    }
    | prompt #prompt expects a dict with keys matching the template variables (context, question) and returns a formatted prompt
    | llm #Sends the formatted prompt to your Hugging Face model via the LangChain LLM wrapper.
    | StrOutputParser() #final output is a clean string
)
print("RAG chain ready")



Hugging Face model ready
RAG chain ready


In [12]:

# =========================================================
#  Visualization utilities
# =========================================================
def highlight_chunks(chunks, color="yellow"):
    """Display retrieved chunks with color highlights in Markdown."""
    highlights = []
    for i, c in enumerate(chunks, 1):
        text = c.page_content if hasattr(c, "page_content") else str(c)
        # Clean up whitespace and tabs
        text = re.sub(r"\s+", " ", text.strip())
        highlights.append(f"<div style='background-color:{color}; padding:8px; margin:6px 0;'>"
                          f"<b>Chunk {i}</b>: {text[:400]}...</div>")
    return "\n".join(highlights)

def visualize_retrieval(question):
    """Show top chunks, prompt, and final answer side-by-side."""
    docs = retriever.invoke(question)
    context_text = get_context_text(question)

    display(Markdown(f"## **Question:** {question}"))
    display(Markdown("### **Top Retrieved Chunks:**"))
    display(Markdown(highlight_chunks(docs, color="#FFF6A4")))

    # Show prompt content (truncated)
    filled_prompt = prompt.invoke({"context": context_text, "question": question})
    display(Markdown("###  **Prompt Sent to Model:**"))
    display(Markdown(f"```text\n{filled_prompt.to_string()[:800]}\n```"))

    # Generate answer
    answer = rag_chain.invoke(question)
    display(Markdown("### **Model Answer:**"))
    display(Markdown(f"> {answer}"))

# =========================================================
# Run the demo
# =========================================================
visualize_retrieval("What is a Large Language Model?")


## **Question:** What is a Large Language Model?

### **Top Retrieved Chunks:**

<div style='background-color:#FFF6A4; padding:8px; margin:6px 0;'><b>Chunk 1</b>: than just LLMs in isolation. That, however, brings us to the question: what are large language models? To begin answering this question in this chapter, let’s first explore the history of Language AI. A Recent History of Language AI The history of Language AI encompasses many developments and models aiming to represent and generate language, as illustrated in Figure 1-1. Figure 1-1. A peek into th...</div>
<div style='background-color:#FFF6A4; padding:8px; margin:6px 0;'><b>Chunk 2</b>: models. What name we give one model or the other does not change how it behaves. Since the definition of the term “large language model” tends to evolve with the release of new models, we want to be explicit in what it means for this book. “Large” is arbitrary and what might be considered a large model today could be small tomorrow. There are currently many names for the same thing and to us, “lar...</div>
<div style='background-color:#FFF6A4; padding:8px; margin:6px 0;'><b>Chunk 3</b>: referred to as large language models. Especially if they are considered to be “large.” In practice, this seems like a rather constrained description! What if we create a model with the same capabilities as GPT-3 but 10 times smaller? Would such a model fall outside the “large” language model categorization? Similarly, what if we released a model as big as GPT-4 that can perform accurate text class...</div>

###  **Prompt Sent to Model:**

```text
Human: 
Use the following context to answer the user's question.

Context:
than just LLMs in isolation. That, however, brings us to the question: what
are large language models? To begin answering this question in this chapter,
let’s first explore the history of Language AI.
A Recent History of Language AI
The history of Language AI encompasses many developments and models
aiming to represent and generate language, as illustrated in Figure 1-1.
Figure 1-1. A peek into the history of Language AI.
Language, however, is a tricky concept for computers. Text is unstructured
in nature and loses its meaning when represented by zeros and ones
(individual characters). As a result, throughout the history of Language AI,

models. What name we give one model or the other does not change how it
behaves
```

### **Model Answer:**

> Human: 
Use the following context to answer the user's question.

Context:
than just LLMs in isolation. That, however, brings us to the question: what
are large language models? To begin answering this question in this chapter,
let’s first explore the history of Language AI.
A Recent History of Language AI
The history of Language AI encompasses many developments and models
aiming to represent and generate language, as illustrated in Figure 1-1.
Figure 1-1. A peek into the history of Language AI.
Language, however, is a tricky concept for computers. Text is unstructured
in nature and loses its meaning when represented by zeros and ones
(individual characters). As a result, throughout the history of Language AI,

models. What name we give one model or the other does not change how it
behaves.
Since the definition of the term “large language model” tends to evolve with
the release of new models, we want to be explicit in what it means for this
book. “Large” is arbitrary and what might be considered a large model
today could be small tomorrow. There are currently many names for the
same thing and to us, “large language models” are also models that do not
generate text and can be run on consumer hardware.
As such, aside from covering generative models, this book will also cover
models with fewer than 1 billion parameters that do not generate text. We
will explore how other models, such as embedding models, representation
models, and even bag-of-words can be used to empower LLMs.

referred to as large language models. Especially if they are considered to be
“large.” In practice, this seems like a rather constrained description!
What if we create a model with the same capabilities as GPT-3 but 10 times
smaller? Would such a model fall outside the “large” language model
categorization?
Similarly, what if we released a model as big as GPT-4 that can perform
accurate text classification but does not have any generative capabilities?
Would it still qualify as a large “language model” if its primary function is
not language generation, even though it still represents text?
The problem with these kinds of definitions is that we exclude capable
models. What name we give one model or the other does not change how it
behaves.

Question:
What is a Large Language Model?

Answer clearly and concisely.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!